**Planning for an Eatery at Scarborough**

**Introduction/Business Problem**

Not every available space is right for a restaurant. A good restaurant location is harder to find than some people think. What may look like the perfect spot- say a bustling pedestrian street in the heart of downtown- may turn out to be a dud.

Of course, food and service are important to the success of a restaurant, but the location can be just as crucial, especially in the early years. 

The following things should be considered while opening a Eatery

Parking
Location
Accessibility
Visibility
Population Base
Know Your Neighbors

With Foursquare Places API, Venue details can be derived. I try to find the best venue location in My neighborhood to open an Eatery. 


**Data and Libraries To be Used**

In [4]:
import numpy as np 

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

##!conda install -c conda-forge geopy --yes 

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

##!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


I shall follow the below Methodologies:
    1. Sourcing Data
    2. Data Preparation
    3. Download the latitude and the longitude coordinates
    4. Explore the Scarborough Area
    5. Identify for nearby locations withing 2 miles 
    6. Identify most promising locations


**Sourcing Data**

In [20]:
from bs4 import BeautifulSoup
datasource = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
sp = BeautifulSoup(datasource, 'lxml')
tab = sp.find_all('table', {'class':'wikitable sortable'})

In [21]:
location = pd.read_html(str(table[0]), index_col=None, header=0)[0]
location.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


**Data Preparation**

In [20]:
#  Get rid of the rows of which Postal Codeas are not assigned

location = location[location.Borough != 'Not assigned']
location.head(5)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [23]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
location.loc[location.Neighbourhood == 'Not assigned', 'Neighbourhood'] = location.Borough
location.head(5)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [22]:
## Filter Data Frame Only for Scarborough Area
location = location[location['Borough']=='Scarborough']
location.head(10)

,Postcode,Borough,Neighbourhood
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern
27,M1C,Scarborough,Highland Creek
28,M1C,Scarborough,Rouge Hill
29,M1C,Scarborough,Port Union
42,M1E,Scarborough,Guildwood
43,M1E,Scarborough,Morningside
44,M1E,Scarborough,West Hill
53,M1G,Scarborough,Woburn
62,M1H,Scarborough,Cedarbrae


In [23]:
# Group Neghborhood by Postcode. Neighbothood under same Borough will be shown as comma separated
Location_Grp = location.groupby(['Postcode','Borough'])['Neighbourhood'].agg(lambda x: ','.join(set(x))).reset_index()

In [24]:
Location_Grp.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park"
7,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge"
8,M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest"
9,M1N,Scarborough,"Cliffside West,Birch Cliff"


In [25]:
#download lattitude and longitude from a separate file
Geo_loc = pd.read_csv('http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
Geo_loc.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [27]:
Geo_loc.rename(columns ={'Postal Code':'Postcode'}, inplace = True) 
Geo_loc.head(2)

,Postcode,Latitude,Longitude
0,M1B,43.806686299999996,-79.19435340000001
1,M1C,43.7845351,-79.16049709999999


**Merge both Data Sets**

In [28]:
Location_Grp.reset_index(drop=True, inplace=True)
Geo_loc.reset_index(drop=True, inplace=True)
Location_Grp = Location_Grp.astype(str)
Geo_loc = Geo_loc.astype(str)
Location_Grp = pd.merge(Location_Grp, Geo_loc, on = 'Postcode')
Location_Grp['Latitude'] = pd.to_numeric(Location_Grp['Latitude'], errors='coerce')
Location_Grp['Longitude'] = pd.to_numeric(Location_Grp['Longitude'], errors='coerce')
Location_Grp.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West,Birch Cliff",43.692657,-79.264848


In [29]:
#Explore the neighborhoods and get the mean loaction
m_lat = Location_Grp.Latitude.mean()
m_lon = Location_Grp.Longitude.mean()
m_lat,m_lon

(43.766228894117646, -79.24908523529412)

In [47]:
from sklearn.cluster import KMeans
import folium # map rendering library

kclusters = 5
loc_grpd_clstr = Location_Grp.drop(['Neighbourhood','Borough','Postcode'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(loc_grpd_clstr)

In [48]:
import matplotlib.cm as cm
import matplotlib.colors as colors

Location_Grp['Cluster Labels'] = kmeans.labels_
map_clstrs = folium.Map(location=[m_lat, m_lon], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Location_Grp['Latitude'], Location_Grp['Longitude'], Location_Grp['Neighbourhood'], Location_Grp['Cluster Labels']):
    label = folium.Popup(str(poi) + ' (Cluster: ' + str(cluster) + ')', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clstrs)

map_clstrs



In [49]:
CLIENT_ID = 'DU43H30KLXB0O2ZXDDDSV5EZILI2QQK3KQ5CPM4Y5MU0M215' # your Foursquare ID
CLIENT_SECRET = 'IOCTTBFVXJ42IZJAZKW3205M3G10KK35JTC1FDICN0RBT1LR' # your Foursquare Secret
VERSION = '20190118' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DU43H30KLXB0O2ZXDDDSV5EZILI2QQK3KQ5CPM4Y5MU0M215
CLIENT_SECRET:IOCTTBFVXJ42IZJAZKW3205M3G10KK35JTC1FDICN0RBT1LR


In [38]:
## Using a radius of 2 Miles to identify for nearby venues

def getNearbyLocs(names, latitudes, longitudes, radius=3300):
    
    locs_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        locs_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_locs = pd.DataFrame([item for venue_list in locs_list for item in venue_list])
    nearby_locs.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_locs)


In [39]:
#radius = 3300
LIMIT = 100
# type your answer here
canada_venues = getNearbyLocs(names=Location_Grp['Postcode'],
                                   latitudes=Location_Grp['Latitude'],
                                   longitudes=Location_Grp['Longitude']
                                  )

M1B
M1C
M1E
M1G
M1H
M1J
M1K
M1L
M1M
M1N
M1P
M1R
M1S
M1T
M1V
M1W
M1X


In [40]:
canada_venues.groupby('Postcode').count()

,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postcode,,,,,,
M1B,58,58,58,58,58,58
M1C,70,70,70,70,70,70
M1E,71,71,71,71,71,71
M1G,100,100,100,100,100,100
M1H,100,100,100,100,100,100
M1J,97,97,97,97,97,97
M1K,100,100,100,100,100,100
M1L,100,100,100,100,100,100
M1M,57,57,57,57,57,57


In [50]:
print('{} uniques categories are there in Scarborough.'.format(len(canada_venues['Venue Category'].unique())))

141 uniques categories are there in Scarborough.


In [51]:
# Explore Trending Venues

# ANALYSIS

# one hot encoding

hotspots = pd.get_dummies(canada_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hotspots['Postcode'] = canada_venues['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [hotspots.columns[-1]] + list(hotspots.columns[:-1])
hotspots = hotspots[fixed_columns]

hotspots.head()



,Postcode,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Beach,Beer Store,Big Box Store,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Laser Tag,Latin American Restaurant,Liquor Store,Lounge,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Store,Neighborhood,Noodle House,Optical Shop,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Pool Hall,Pub,Ramen Restaurant,Restaurant,Rock Climbing Spot,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Smoothie Shop,Snack Place,Spa,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Toy / Game Store,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Zoo,Zoo Exhibit
0,M1B,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,M1B,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M1B,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,M1B,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,M1B,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [52]:
spot_grpd = hotspots.groupby('Postcode').mean().reset_index()
spot_grpd.head()

,Postcode,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Beach,Beer Store,Big Box Store,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Laser Tag,Latin American Restaurant,Liquor Store,Lounge,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Store,Neighborhood,Noodle House,Optical Shop,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Pool Hall,Pub,Ramen Restaurant,Restaurant,Rock Climbing Spot,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Smoothie Shop,Snack Place,Spa,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Toy / Game Store,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Zoo,Zoo Exhibit
0,M1B,0.00,0.0,0.0,0.017241,0.0,0.0,0.0,0.000000,0.017241,0.0,0.0,0.017241,0.0,0.0,0.00,0.0,0.017241,0.0,0.0,0.034483,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,0.017241,0.0,0.00,0.017241,0.00,0.0,0.00,0.086207,0.0,0.000000,0.000000,0.0,0.00,0.0,0.017241,0.00,0.017241,0.00,0.0,0.000000,0.0,0.0,0.0,0.051724,0.0,0.00,0.0,0.0,0.000000,0.00,0.0,0.017241,0.00,0.0,0.0,0.0,0.034483,0.000000,0.000000,0.034483,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.017241,0.00,0.000000,0.00,0.0,0.0,0.017241,0.00,0.00,0.0,0.000000,0.0,0.0,0.00,0.00,0.017241,0.00,0.0,0.017241,0.051724,0.0,0.051724,0.068966,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.034483,0.00,0.00,0.0,0.0,0.000000,0.0,0.017241,0.00,0.000000,0.00,0.00,0.017241,0.00,0.0,0.0,0.00,0.000000,0.00,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00,0.034483,0.241379
1,M1C,0.00,0.0,0.0,0.028571,0.0,0.0,0.0,0.014286,0.028571,0.0,0.0,0.028571,0.0,0.0,0.00,0.0,0.042857,0.0,0.0,0.042857,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.014286,0.0,0.00,0.014286,0.00,0.0,0.00,0.114286,0.0,0.014286,0.014286,0.0,0.00,0.0,0.000000,0.00,0.028571,0.00,0.0,0.000000,0.0,0.0,0.0,0.057143,0.0,0.00,0.0,0.0,0.014286,0.00,0.0,0.042857,0.00,0.0,0.0,0.0,0.000000,0.000000,0.028571,0.042857,0.014286,0.014286,0.0,0.00,0.0,0.014286,0.0,0.0,0.0,0.0,0.00,0.0,0.014286,0.000000,0.0,0.014286,0.00,0.000000,0.00,0.0,0.0,0.014286,0.00,0.00,0.0,0.014286,0.0,0.0,0.00,0.00,0.014286,0.00,0.0,0.000000,0.071429,0.0,0.028571,0.071429,0.014286,0.0,0.0,0.014286,0.0,0.014286,0.0,0.042857,0.00,0.00,0.0,0.0,0.014286,0.0,0.000000,0.00,0.014286,0.00,0.00,0.028571,0.00,0.0,0.0,0.00,0.000000,0.00,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.000000
2,M1E,0.00,0.0,0.0,0.014085,0.0,0.0,0.0,0.000000,0.028169,0.0,0.0,0.056338,0.0,0.0,0.00,0.0,0.028169,0.0,0.0,0.042254,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.00,0.0,0.00,0.140845,0.0,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.00,0.042254,0.00,0.0,0.014085,0.0

In [53]:
#Identifying the top 10 venue by Borough (i.e. Postcode)

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [54]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = hotspots['Postcode']

for ind in np.arange(hotspots.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hotspots.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Zoo Exhibit,Discount Store,Farm,Falafel Restaurant,Event Space,Electronics Store,Dumpling Restaurant,Dog Run,Diner,Filipino Restaurant
1,M1B,Athletics & Sports,Zoo Exhibit,Dog Run,Fast Food Restaurant,Farm,Falafel Restaurant,Event Space,Electronics Store,Dumpling Restaurant,Discount Store
2,M1B,Zoo,Dog Run,Fast Food Restaurant,Farm,Falafel Restaurant,Event Space,Electronics Store,Dumpling Restaurant,Discount Store,Fish & Chips Shop
3,M1B,Zoo,Dog Run,Fast Food Restaurant,Farm,Falafel Restaurant,Event Space,Electronics Store,Dumpling Restaurant,Discount Store,Fish & Chips Shop
4,M1B,Zoo Exhibit,Discount Store,Farm,Falafel Restaurant,Event Space,Electronics Store,Dumpling Restaurant,Dog Run,Diner,Filipino Restaurant
